In [ ]:
%run ./define_tables

In [0]:
#  bronze london authority data
columns = ['site', 'longitude', 'latitude']
locations = spark.createDataFrame(
    [
        ('IS2', -0.116146000000, 51.555378000000), 
        ('EN4', 51.614864007000, -0.050765836369),
        ('GB6', 51.456300000000, 0.0856060000000)
    ], 
    columns
)

locations.write.mode("overwrite").saveAsTable(bronze_london_authorities_table)




In [0]:
# autoloader (CCD)_data_folder)
environmental_data = (
    spark.readStream.format("cloudFiles")
    .option("cloudFiles.schemaLocation", "/tmp/schemaLocation")
    .option("cloudFiles.format", "csv")
    .option("cloudFiles.includeExistingFiles", "true")  
    .option("header", "true")
    .load(environmental_data_folder)
)

# change column name 'Provisional or Ratified'
environmental_data = environmental_data.withColumnRenamed(
    'Provisional or Ratified', 'Provisional_or_Ratified'
)

# Start the stream
query =  environmental_data.writeStream.format("delta") \
    .outputMode("append") \
    .option("checkpointLocation", checkpoint_path) \
    .option("mergeSchema", "true") \
    .trigger(availableNow=True) \
    .toTable(bronze_table)

# Check that rows are written
# Wait for the stream to finish
query.awaitTermination()
display(spark.table(bronze_table).limit(5))

